In [20]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [21]:
glass = pd.read_csv('glass.csv')
glass.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516523,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [22]:
# Dividir o dados para treinamento e teste
train, test = train_test_split(glass, test_size=.4, random_state=123)

print('train {0}'.format(train.shape))
print('test {0}'.format(test.shape))

X_train = train.drop(['Type'], axis=1)
y_train = train['Type']

X_test = test.drop(['Type'], axis=1)
y_test = test['Type']

print('X_train {0}'.format(X_train.shape))
print('y_train {0}'.format(y_train.shape))
print('X_test {0}'.format(X_test.shape))
print('y_test {0}'.format(y_test.shape))

train (128, 10)
test (86, 10)
X_train (128, 9)
y_train (128,)
X_test (86, 9)
y_test (86,)


In [23]:
model = XGBClassifier()
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.71      0.74      0.72        23
           2       0.67      0.90      0.76        29
           3       0.00      0.00      0.00         6
           5       0.75      0.38      0.50         8
           6       1.00      0.50      0.67         4
           7       0.81      0.81      0.81        16

   micro avg       0.71      0.71      0.71        86
   macro avg       0.66      0.55      0.58        86
weighted avg       0.68      0.71      0.68        86



In [24]:
# K-fold cross validation
# k=10 sobre o set de treinamento, bagging
X = train.drop(['Type'], axis=1)
y = train.Type
scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')
print('accuracy:', scores)
print('avg accuracy:', scores.mean())

/home/jecs89/anaconda3/envs/tutorialConda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


accuracy: [0.75       0.73333333 0.73333333 0.78571429 0.71428571 0.5
 0.75       0.7        0.6        0.6       ]
avg accuracy: 0.6866666666666666


In [25]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
# avaliando o algoritmo
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.67      0.70      0.68        23
           2       0.68      0.72      0.70        29
           3       0.40      0.33      0.36         6
           5       0.83      0.62      0.71         8
           6       0.60      0.75      0.67         4
           7       0.93      0.88      0.90        16

   micro avg       0.71      0.71      0.71        86
   macro avg       0.69      0.67      0.67        86
weighted avg       0.71      0.71      0.71        86



In [26]:
# K-fold cross validation
# k=10 sobre o set de treinamento, clf
X = train.drop(['Type'], axis=1)
y = train.Type
scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
print('accuracy:', scores)
print('avg accuracy:', scores.mean())

accuracy: [0.75       0.46666667 0.8        0.64285714 0.42857143 0.33333333
 0.66666667 0.7        0.6        0.5       ]
avg accuracy: 0.5888095238095238


/home/jecs89/anaconda3/envs/tutorialConda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
